In [3]:
from pyspark import SparkFiles
from pyspark import SparkContext 
from pyspark import SQLContext
import sys
import os

In [5]:
sc = SparkContext(appName ='First Lesson', master = 'local')
sc

<SparkContext master=local appName=First Lesson>

In [6]:
rdd = sc.textFile('file:///Users/tjmask/Desktop/test.txt')

In [7]:
rdd.take(2)

['USE db_countries;', '-- IMPORTING DATA FROM A CSV']

In [8]:
def fuc(lines):
    lines =lines.lower()
    lines = lines.split()
    return lines

In [9]:
rdd1 = rdd.map(fuc)

In [10]:
rdd1.take(2)

[['use', 'db_countries;'], ['--', 'importing', 'data', 'from', 'a', 'csv']]

In [12]:
rdd2 = rdd.flatMap(fuc)
rdd2.take(5)

['use', 'db_countries;', '--', 'importing', 'data']

In [15]:
stopwords=['a', 'use', 'the', 'is']
rdd3 = rdd2.filter(lambda x: x not in stopwords)
rdd3.take(2)

['db_countries;', '--']